In [1]:
import sys; sys.path.append("..")
from epi_model.common_imports import *

In [2]:
class early_infect_analysis:
    def __init__(self, sim):
        from random import choice
        
        # run 50 simulations...
        NN = 50

        for i in range(NN):
            sim.init_attributes()
            
            if (i+1)%(NN//10)==0:
                print(i,"/",NN)
            sim.state_change(choice(range(sim.tnet.Nnodes)), 'inf')
            sim.run(20)
            
        all_infect_t = inf_times = [
            ( sc[1], sc[0] )
            for sc in sim.state_changes 
            if sc[0] != 0 and\
            sc[2] == 'inf'
        ]
        
        self.means = {
            n: np.mean([t for nn,t in all_infect_t if nn==n])
            for n in range(sim.tnet.Nnodes)
        }
        self.counts = {
            n: np.sum([1 for nn,t in all_infect_t if nn==n])
            for n in range(sim.tnet.Nnodes)
        }

def early_infect(sim, vaccinateN=100):
    if not hasattr(sim, 'early_infect'):
        sim.early_infect = early_infect_analysis(sim)
    
    top100a = sorted(range(sim.tnet.Nnodes), key=lambda i:-sim.early_infect.counts[i])[:vaccinateN*2]
    top100b = sorted(top100a, key=lambda i:sim.early_infect.means[i])
    tovaccinate = top100b[:vaccinateN]

    return tovaccinate

In [3]:
from epi_model import *

In [4]:
tnet = temporalNetwork.load('high school')

In [5]:
p = dict(params.covid_estimate)
p['s2e'] = params.daily_to_momentary(tnet, 0.2)
sim = simulations.SEIR_daily(tnet, p)

In [6]:
list_len = 156

In [7]:
# run 50 simulations...
NN = 50

to_vaccinate = []

while len(to_vaccinate) < list_len:

    all_infect_t = []
    for i in range(NN):
        sim.init_attributes()

        if (i+1)%(NN//10)==0:
            print(i,"/",NN)
            
        for vacc_i in to_vaccinate:
            sim.state_change(inf_i, 'vacc')
            
        for inf_i in sample([x for x in range(sim.tnet.Nnodes) if not sim.vacc[x]], 20):
            sim.state_change(inf_i, 'inf')
            
        sim.run(100)
        
        inf_times = [
            ( sc[1], sc[0] )
            for sc in sim.state_changes 
            if sc[0] != 0 and\
            sc[2] == 'inf'
        ]
        all_infect_t += inf_times

   
    means = {
        n: np.mean([t for nn,t in all_infect_t if nn==n])
        for n in range(sim.tnet.Nnodes)
    }
    counts = {
        n: np.sum([1 for nn,t in all_infect_t if nn==n])
        for n in range(sim.tnet.Nnodes)
    }

    q75 = np.quantile(list(counts.values()), 0.75)
    lim1 = [x for x,y in counts.items() if y >= q75]
    lim2 = sorted(lim1, key=lambda x:means[x])[:5]

    to_vaccinate += lim2

4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50


c:\users\amcga\envs\citation-deaths\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\amcga\envs\citation-deaths\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44 / 50
49 / 50
4 / 50
9 / 50
14 / 50
19 / 50
24 / 50
29 / 50
34 / 50
39 / 50
44

In [8]:
with open('2.0 iterative.pickle', 'wb') as outf:
    pickle.dump(to_vaccinate, outf)